In [25]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import toolkitEngineering
import importlib
importlib.reload(toolkitEngineering)

<module 'toolkitEngineering' from 'd:\\Projetos\\ETL-ML-Dengue\\src\\engineering\\toolkitEngineering.py'>

In [26]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_sinan_bronze = os.path.join(diretorio_bronze, 'sinan')
diretorio_sinan_bronze_csv = os.path.join(diretorio_sinan_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_sinan_silver = os.path.join(diretorio_silver, 'sinan')
diretorio_sinan_silver_tsv = os.path.join(diretorio_sinan_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_sinan_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_sinan_silver_tsv), exist_ok=True)




In [27]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_sinan_bronze_csv) if arquivo.endswith("csv")]
arquivos_csv = arquivos_csv[-2:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['head_100_DENGBR10.tsv', 'head_100_DENGBR11.tsv']

In [28]:
# Função para converter NU_IDADE_N para idade em anos
def convert_to_years(age):
    if pd.isnull(age):
        return np.nan
    if len(str(age)) < 4:
        return np.nan
    age_str = str(age)
    prefix = age_str[0]
    value = int(age_str[1:])
    
    if prefix == '1':  # Hora
        return value / (24 * 365)  # Aproximação para converter horas em anos
    elif prefix == '2':  # Dia
        return value / 365  # Aproximação para converter dias em anos
    elif prefix == '3':  # Mês
        return value / 12  # Aproximação para converter meses em anos
    elif prefix == '4':  # Ano
        return value
    else:
        return np.nan



In [30]:
colunas_importantes = [
    "NU_ANO","NU_MES", "ID_MUNICIP", "DT_NOTIFIC", "SEM_NOT", "DT_SIN_PRI", "SEM_PRI", "NU_IDADE_N", "IDADE_ANOS", "CS_SEXO",
    "CS_GESTANT", "CS_RACA", "CS_ESCOL_N", "SG_UF", "ID_MN_RESI", "ID_PAIS", "NDUPLIC_N", "DT_INVEST", "DT_SORO",
    "DT_NS1", "RESUL_NS1", "DT_VIRAL", "RESUL_VI_N", "DT_PCR", "RESUL_PCR_", "SOROTIPO", "HISTOPA_N", "IMUNOH_N",
    "TPAUTOCTO", "COUFINF", "COPAISINF", "COMUNINF", "EVOLUCAO", "DT_OBITO", "DT_ENCERRA", "MANI_HEMOR", "EPISTAXE",
    "PETEQUIAS", "HEMATURA", "SANGRAM", "LACO_N", "PLASMATICO", "EVIDENCIA", "COMPLICA", "HOSPITALIZ", "DT_INTERNA",
    "CRITERIO","CLASSI_FIN","RESUL_SORO","METRO","GENGIVO","UF","MUNICIPIO"
]
column_map = {
    "DT_INTER": "DT_INTERNA"
}

    # Lista das colunas que queremos tentar converter para inteiros
columns_to_convert = [
    "CS_GESTANT", "CS_RACA", "ID_PAIS", "NDUPLIC_N",
    "RESUL_SORO", "RESUL_NS1", "RESUL_VI_N", "RESUL_PCR_", "SOROTIPO",
    "HISTOPA_N", "IMUNOH_N", "CLASSI_FIN", "TPAUTOCTO", "EVOLUCAO",
    "MANI_HEMOR", "EPISTAXE", "GENGIVO", "METRO", "PETEQUIAS",
    "HEMATURA", "SANGRAM", "LACO_N", "PLASMATICO", "EVIDENCIA",
    "COMPLICA", "HOSPITALIZ", "UF", "CS_ESCOL_N", "CRITERIO", "COUFINF", "MUNICIPIO", "COMUNINF", "ID_MN_RESI"]

In [31]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_sinan_bronze_csv, arquivo_csv), sep=',')
    # Trocar "NA" por np.nan (ou None, ambos representam valores nulos em Pandas)
    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    # Remover aspas das colunas que são de tipo string
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    # Aplicar a função e criar a nova coluna IDADE_ANOS
    df['IDADE_ANOS'] = df['NU_IDADE_N'].apply(convert_to_years)
    df['NU_MES'] = df['DT_NOTIFIC'].apply(lambda x: x.split('-')[1].lstrip("0"))
    df = df.rename(columns=column_map)
    df = df[colunas_importantes]


    # Itera sobre cada coluna e tenta converter para inteiro
    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

    df.columns = df.columns.str.lower()
    

    df.to_csv(os.path.join(diretorio_sinan_silver_tsv, arquivo_tsv), sep='\t', index=False)

In [32]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitEngineering.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_sinan_silver_tsv, arquivo_tsv)
    toolkitEngineering.inserir_no_banco(cursor, conn, caminho_arquivo, 'sinan_dengue', delimiter='\t',
                                        diretorio_dotenv=diretorio_dotenv)
